# BI Metrics in Power BI vs. SQL

The aim of this notebook is to demonstrate the savings we have in productivity when we use DAX in Power BI (or SSAS) to calculate complex metrics, instead of T-SQL (e.g. YoY and YTD metrics). For this reason we're going to use the [World Wide Importers DW](https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-dw-install-configure?view=sql-server-ver15). I've installed SQL Server on Ubuntu using the instructions [here](https://docs.microsoft.com/en-us/sql/linux/quickstart-install-connect-ubuntu?view=sql-server-ver15), and Azure Data Studio using the instructions [here](https://docs.microsoft.com/en-us/sql/azure-data-studio/download-azure-data-studio?view=sql-server-ver15). Then imported the database as a .bak file, and created this notebook in Azure Data Studio. At the same time, I've used Power BI Desktop on a Windows VM, and imported all relevant tables there:

In [1]:
use WideWorldImportersDW

select schema_name(t.schema_id) as schema_name,
       t.name as table_name
from sys.tables t
where schema_name(t.schema_id) IN ('Dimension','Fact')
order by t.schema_id,table_name;

(14 rows affected)

Total execution time: 00:00:00.111

schema_name,table_name
Dimension,City
Dimension,Customer
Dimension,Date
Dimension,Employee
Dimension,Payment Method
Dimension,Stock Item
Dimension,Supplier
Dimension,Transaction Type
Fact,Movement
Fact,Order


Power BI will detect the relationships automatically, as there are foreign keys defined in the database. The part of the data model which is relevant to our example is the below:

![](./files/modelpart.png)

We also need to mark the 'Dimension.Date' table as a 'Date' table in Power BI:

![](./files/MarkDate.png)

and sort months by month number:

![](./files/SortMonths.png)

To calculate YTD (Year To Date) sales per brand for each month of 2014, we'd like something like this:

In [2]:
select 
 Color
,[Month]
,FORMAT(SUM(Revenue) OVER (PARTITION BY color  ORDER BY [Month No] ), '##,##0.##') AS [YTD Sales per Color] 
,FORMAT(Revenue,'##,##0.##')  [Sales per Color]--for ref
FROM

(select P.[Color] Color
,D.[Calendar Month Label] [Month]
,D.[Calendar Month Number] [Month No]
,sum(F.[Total Excluding Tax]) Revenue

from 
fact.Sale F 
JOIN dimension.[Stock Item] P on F.[Stock Item Key]=P.[Stock Item Key]
JOIN dimension.[Date] D on F.[Invoice Date Key]=D.[Date]

where year([invoice date key])=2014

group by P.[Color], D.[Calendar Month Number],D.[Calendar Month Label]
) NESTED

order by color,[Month No]

(96 rows affected)

Total execution time: 00:00:00.145

Color,Month,YTD Sales per Color,Sales per Color
Black,CY2014-Jan,"598,457","598,457"
Black,CY2014-Feb,"1,192,999","594,542"
Black,CY2014-Mar,"1,752,944","559,945"
Black,CY2014-Apr,"2,381,596","628,652"
Black,CY2014-May,"3,085,738","704,142"
Black,CY2014-Jun,"3,790,080","704,342"
Black,CY2014-Jul,"4,553,509","763,429"
Black,CY2014-Aug,"5,167,294","613,785"
Black,CY2014-Sep,"5,755,181","587,887"
Black,CY2014-Oct,"6,394,293","639,112"



If we'd like to so the same for 2015, we'd need to change the where clause. If we'd like to group for another attribute, we'd need to replace [Stock Item].[Color] with that attribute (e.g. [Stock Item].Brand). If the attribute belonged to another dimension, we'd have to change the query completely.


On the other hand, on Power BI, we'd only need to define this DAX measure: `Sales YTD = TOTALYTD([Sales],'Dimension Date'[Date])`

having defined measure `Sales = SUM('Fact Sale'[Total Excluding Tax])` to improve readability.

Then we could create any report we'd like with YTD sales, for any year, for any customer, for any product... That would be achieved by simple drag-and-drop, for any of these attributes and any visual. We could also add filters and groupings at will. For example two reports showing YTD sales per color and brand could be the following:

![](./files/YTD_Color.png)

![](./files/YTD_Brand.png)

Similarly, to calculate YoY (Year on Year) percentage sales increase between 2014 and 2013, we would do something like this:

In [3]:
select 
Color
,(sales - lag1)/lag1 as [YoY% Sales]

FROM (
        select 
        Color
        ,[Invoice Year]
        ,Revenue as Sales
        ,Lag(Revenue,1,0) OVER (PARTITION BY color ORDER BY [Invoice Year]) as Lag1

        FROM (

            select P.[Color] Color
            ,year(D.Date) as [Invoice Year]
            ,sum(F.[Total Excluding Tax]) Revenue

            from 
            fact.Sale F 
            JOIN dimension.[Stock Item] P on F.[Stock Item Key]=P.[Stock Item Key]
            JOIN dimension.[Date] D on F.[Invoice Date Key]=D.[Date]

            where year([invoice date key]) IN (2013,2014)

            group by P.[Color], year(D.Date)
        ) NESTED
    ) NESTED1
where [Invoice Year]=2014
order by color

(8 rows affected)

Total execution time: 00:00:00.642

Color,YoY% Sales
Black,0.108202
Blue,0.082444
Gray,0.098283
Light Brown,0.129021
N/A,0.097150
Red,0.063739
White,0.066177
Yellow,0.029636


The DAX measure is also simple: `Sales YoY% = DIVIDE(\[Sales\]-\[Sales Last Year\],\[Sales Last Year\])\`
also having defined `Sales Last Year = CALCULATE([Sales],SAMEPERIODLASTYEAR('Dimension Date'[date]))` for readability.

and can be used to create reports involving additional attributes, without requiring other code:

![](./files/YoY_Color.png)

![](./files/YoY_Brand.png)

It should be clear by now that, using just a couple of DAX measures, we can create any sort of visualizations, filtering and grouping by any attribute we want. Even if we add dimensions and attributes in our model, DAX measures we have already created will work for the new attributes as well. In SQL we would need to create additional queries for each and every filter / group combination we might need. 

  

Consequently, using the correct tool for the correct problem, we can save significantly amount of development time and, as a matter of fact, improve execution performance. At the same time we enable self service reporting, as the person who designs the report or performs an analysis using visualizations, does not need to be the same person who wites the DAX.